In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [3]:
customer_id = '5'
formatted_attribute = 'Indoor_outdoor'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_') 

# buckets = """Window Blinds"""
buckets = """Package Quantity"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/Delete_Motion_Max_Min_temp_date.sql', params=params) 
# df=df[df['value']=='n/a']
print(len(df)) 
df.head(1) 

224355


attributes snake_case_name                                       name  \
0  Indoor/Outdoor  indoor_outdoor  Arctic X Heated Car Seat Cushion in Black   

  external_id  \
0    62221887   

                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0  Winter driving will be a pleasure with this Arctic X Heated Car Seat Cushion. While your car is warming up, you can have a warm comfy seat in minutes. Just like having a luxury car, you can select between 2 settings for your own personal comfort. Select between 2 settings for your own personal comfort level Easily plugs into your car's 12V socket Polyester/sponge/copper/wire/plastic 54\" cord length Measures 39\" L x 19\" W x 1\" D Imported   

  curated_date  customers_id           buckets value   family_id resolution  
0   2021-08-14             5  Seating Cushions   n/a  1062221887   standard

In [13]:
regex_pattern_final_regular = r"(?i)(water.{0,30}resistant)|(water.{0,30}proof)|(water.{0,30}repellant)|()" 
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x))) 
regex_pattern_rug = r"(?i)(rug)|()" 
df['rug_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x))) 

In [18]:
outdoor=df[(df['matches'].astype(str)!='[]')(df['buckets'].astype(str)!='Bean Bag Chairs')&(df['buckets'].astype(str)!='Covers/Protectors')&(df['buckets'].astype(str)!='Office Chairs')&(df['buckets'].astype(str)!='Tablecloths & Table Runners')&(df['buckets'].astype(str)!='Curtains')&(df['value'].astype(str)=='n/a')&(df['rug_matches'].astype(str)!='[]')&(df['buckets'].astype(str)!='Rugs & Mats B')&(df['buckets'].astype(str)!='Rugs & Mats')]
outdoor

attributes snake_case_name  \
1       Indoor/Outdoor  indoor_outdoor   
2       Indoor/Outdoor  indoor_outdoor   
3       Indoor/Outdoor  indoor_outdoor   
2560    Indoor/Outdoor  indoor_outdoor   
2561    Indoor/Outdoor  indoor_outdoor   
2562    Indoor/Outdoor  indoor_outdoor   
2927    Indoor/Outdoor  indoor_outdoor   
3194    Indoor/Outdoor  indoor_outdoor   
3201    Indoor/Outdoor  indoor_outdoor   
3516    Indoor/Outdoor  indoor_outdoor   
3517    Indoor/Outdoor  indoor_outdoor   
4324    Indoor/Outdoor  indoor_outdoor   
4325    Indoor/Outdoor  indoor_outdoor   
4840    Indoor/Outdoor  indoor_outdoor   
4843    Indoor/Outdoor  indoor_outdoor   
5127    Indoor/Outdoor  indoor_outdoor   
6707    Indoor/Outdoor  indoor_outdoor   
7314    Indoor/Outdoor  indoor_outdoor   
7315    Indoor/Outdoor  indoor_outdoor   
7501    Indoor/Outdoor  indoor_outdoor   
7546    Indoor/Outdoor  indoor_outdoor   
7659    Indoor/Outdoor  indoor_outdoor   
8101    Indoor/Outdoor  indoor_outdoor   
8143    Indoor/Outdoor  indoor_outdoor   
8330    Indoor/Outdoor  indoor_outdoor   
8435    Indoor/Outdoor  indoor_outdoor   
8463    Indoor/Outdoor  indoor_outdoor   
8475    Indoor/Outdoor  indoor_outdoor   
8557    Indoor/Outdoor  indoor_outdoor   
8558    Indoor/Outdoor  indoor_outdoor   
8559    Indoor/Outdoor  indoor_outdoor   
8612    Indoor/Outdoor  indoor_outdoor   
8634    Indoor/Outdoor  indoor_outdoor   
8659    Indoor/Outdoor  indoor_outdoor   
8702    Indoor/Outdoor  indoor_outdoor   
8704    Indoor/Outdoor  indoor_outdoor   
8705    Indoor/Outdoor  indoor_outdoor   
8709    Indoor/Outdoor  indoor_outdoor   
8710    Indoor/Outdoor  indoor_outdoor   
8711    Indoor/Outdoor  indoor_outdoor   
8712    Indoor/Outdoor  indoor_outdoor   
8713    Indoor/Outdoor  indoor_outdoor   
8723    Indoor/Outdoor  indoor_outdoor   
8762    Indoor/Outdoor  indoor_outdoor   
9180    Indoor/Outdoor  indoor_outdoor   
9301    Indoor/Outdoor  indoor_outdoor   
9307    Indoor/Outdoor  indoor_outdoor   
9572    Indoor/Outdoor  indoor_outdoor   
9620    Indoor/Outdoor  indoor_outdoor   
9621    Indoor/Outdoor  indoor_outdoor   
9622    Indoor/Outdoor  indoor_outdoor   
9623    Indoor/Outdoor  indoor_outdoor   
9624    Indoor/Outdoor  indoor_outdoor   
9625    Indoor/Outdoor  indoor_outdoor   
9626    Indoor/Outdoor  indoor_outdoor   
9627    Indoor/Outdoor  indoor_outdoor   
9628    Indoor/Outdoor  indoor_outdoor   
9629    Indoor/Outdoor  indoor_outdoor   
9630    Indoor/Outdoor  indoor_outdoor   
9631    Indoor/Outdoor  indoor_outdoor   
9632    Indoor/Outdoor  indoor_outdoor   
9724    Indoor/Outdoor  indoor_outdoor   
9743    Indoor/Outdoor  indoor_outdoor   
9746    Indoor/Outdoor  indoor_outdoor   
9747    Indoor/Outdoor  indoor_outdoor   
9748    Indoor/Outdoor  indoor_outdoor   
9752    Indoor/Outdoor  indoor_outdoor   
9753    Indoor/Outdoor  indoor_outdoor   
9755    Indoor/Outdoor  indoor_outdoor   
9758    Indoor/Outdoor  indoor_outdoor   
9760    Indoor/Outdoor  indoor_outdoor   
9761    Indoor/Outdoor  indoor_outdoor   
9762    Indoor/Outdoor  indoor_outdoor   
9763    Indoor/Outdoor  indoor_outdoor   
9764    Indoor/Outdoor  indoor_outdoor   
9766    Indoor/Outdoor  indoor_outdoor   
9767    Indoor/Outdoor  indoor_outdoor   
9769    Indoor/Outdoor  indoor_outdoor   
9856    Indoor/Outdoor  indoor_outdoor   
9857    Indoor/Outdoor  indoor_outdoor   
9858    Indoor/Outdoor  indoor_outdoor   
9859    Indoor/Outdoor  indoor_outdoor   
9862    Indoor/Outdoor  indoor_outdoor   
9863    Indoor/Outdoor  indoor_outdoor   
9864    Indoor/Outdoor  indoor_outdoor   
9890    Indoor/Outdoor  indoor_outdoor   
9965    Indoor/Outdoor  indoor_outdoor   
9966    Indoor/Outdoor  indoor_outdoor   
9967    Indoor/Outdoor  indoor_outdoor   
9971    Indoor/Outdoor  indoor_outdoor   
9988    Indoor/Outdoor  indoor_outdoor   
10190   Indoor/Outdoor  indoor_outdoor   
10197   Indoor/Outdoor  indoor_outdoor   
10513   Indoor/Outdoor  indoor_outdoor   
10520   Indoor/Outdoor

In [20]:
outdoor[curation_col]='Outdoor'
outdoor_new=outdoor[['external_id',curation_col]]
outdoor_new

external_id Q:indoor_outdoor
1         65964019          Outdoor
2         65964064          Outdoor
3         65963975          Outdoor
2560      66329572          Outdoor
2561      66329565          Outdoor
2562      66329541          Outdoor
2927      61494022          Outdoor
3194      69700270          Outdoor
3201      41689660          Outdoor
3516      69648469          Outdoor
3517      20720902          Outdoor
4324      63191011          Outdoor
4325      63190977          Outdoor
4840      64649436          Outdoor
4843      64649450          Outdoor
5127      64636474          Outdoor
6707      69639136          Outdoor
7314      61494091          Outdoor
7315      61494152          Outdoor
7501      63398557          Outdoor
7546      69611411          Outdoor
7659      69093746          Outdoor
8101      40655642          Outdoor
8143      61494350          Outdoor
8330      69594833          Outdoor
8435      15080396          Outdoor
8463      69701010          Outdoor
8475      45939891          Outdoor
8557      46689467          Outdoor
8558      46689474          Outdoor
8559      46689481          Outdoor
8612      65749302          Outdoor
8634      43086498          Outdoor
8659      67203956          Outdoor
8702      41494929          Outdoor
8704      67030804          Outdoor
8705      43769223          Outdoor
8709      69594718          Outdoor
8710      69594728          Outdoor
8711      69594812          Outdoor
8712      69594834          Outdoor
8713      69594845          Outdoor
8723      69507838          Outdoor
8762      69590498          Outdoor
9180      63566765          Outdoor
9301      61651708          Outdoor
9307      63862201          Outdoor
9572      61420977          Outdoor
9620      69629497          Outdoor
9621      69629498          Outdoor
9622      69629496          Outdoor
9623      69629499          Outdoor
9624      69629493          Outdoor
9625      69629494          Outdoor
9626      69629492          Outdoor
9627      69629495          Outdoor
9628      69629502          Outdoor
9629      69629503          Outdoor
9630      69629501          Outdoor
9631      69629504          Outdoor
9632      69629500          Outdoor
9724      46690142          Outdoor
9743      60549211          Outdoor
9746      60549273          Outdoor
9747      60550026          Outdoor
9748      60549365          Outdoor
9752      60549488          Outdoor
9753      60549549          Outdoor
9755      60549570          Outdoor
9758      60549518          Outdoor
9760      60549662          Outdoor
9761      60549693          Outdoor
9762      60550057          Outdoor
9763      60549723          Outdoor
9764      60549754          Outdoor
9766      60549785          Outdoor
9767      60549815          Outdoor
9769      60549846          Outdoor
9856      63862126          Outdoor
9857      63862133          Outdoor
9858      63862119          Outdoor
9859      63862157          Outdoor
9862      63862171          Outdoor
9863      63862188          Outdoor
9864      63862195          Outdoor
9890      65729984          Outdoor
9965      64636436          Outdoor
9966      64649429          Outdoor
9967      64649467          Outdoor
9971      65693667          Outdoor
9988      44314408          Outdoor
10190     45939884          Outdoor
10197     40383361          Outdoor
10513     62071475          Outdoor
10520     62681964          Outdoor
10521     62681971          Outdoor
10522     40938806          Outdoor
10523     62681988          Outdoor
10525     62681995          Outdoor
10526     62681957          Outdoor
10527     62681940          Outdoor
10528     62681919          Outdoor
15320     66329114          Outdoor
17286     61494367          Outdoor
19986     67408641          Outdoor
20366     60422668          Outdoor
20367     60422675          Outdoor
20421     62880442          Outdoor
20422     60294944          Outdoor
20427     60294937          Outdoor
20527     6

In [10]:
outdoor

attributes snake_case_name  \
1       Indoor/Outdoor  indoor_outdoor   
2       Indoor/Outdoor  indoor_outdoor   
3       Indoor/Outdoor  indoor_outdoor   
183     Indoor/Outdoor  indoor_outdoor   
2560    Indoor/Outdoor  indoor_outdoor   
...                ...             ...   
224113  Indoor/Outdoor  indoor_outdoor   
224121  Indoor/Outdoor  indoor_outdoor   
224126  Indoor/Outdoor  indoor_outdoor   
224127  Indoor/Outdoor  indoor_outdoor   
224128  Indoor/Outdoor  indoor_outdoor   

                                                                                        name  \
1        Classic Accessories® Montlake 19-Inch x 20-Inch Patio Back Cushion Replacement Foam   
2        Classic Accessories® Montlake 23-Inch x 21-Inch Patio Seat Cushion Replacement Foam   
3       Classic Accessories® Montlake 54-Inch x 18-Inch Patio Bench Cushion Replacement Foam   
183                               GelPro® Classic Basketweave 20\" x 32\" Floor Mat in Khaki   
2560                                           TravelChair® Company Big Bubba Chair in Black   
...                                                                                      ...   
224113         Greendale Home Fashions Hyatt 2-Piece Jumbo Rocking Chair Cushion Set in Moss   
224121                   Greendale Home Fashions Hyatt 4-Piece Nook Cushion Set in Chocolate   
224126            Greendale Home Fashions Hyatt Microfiber Chair Pads in Burgundy (Set of 2)   
224127               Greendale Home Fashions Hyatt Microfiber Chair Pads in Cream (Set of 2)   
224128             Greendale Home Fashions Hyatt Microfiber Chair Pads in Scarlet (Set of 2)   

       external_id  \
1         65964019   
2         65964064   
3         65963975   
183       42740605   
2560      66329572   
...            ...   
224113    69639148   
224121    69639144   
224126    69639134   
224127    69639135   
224128    69639138   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
1                                                                                                                                                                                                                                    Ensure you're always sitting in the lap of comfort with Replacement Back Cushion Foam from Classic Accessories. This foam is long-lasting and durable, with a resilient three layer design that conforms to your shape and offers superior comfort. Replace your aging, worn back cushion with this durable, long-lasting foam Features a 3-layered design that includes outer layers of open-cell foam enclosing a high density core, which offers resilience but conforms to your shape The sewn casing provides a clean finish for the interior and encloses raw edges to protect your foam Water-resistant Foam Hand wash only Measures 20\" L x 19\" W x 4\" H 1-year limited manufacturer's warranty Imported   
2                                                                                                                                                                                                                                    Ensure you'r

In [ ]:
error----Switch Max and Min when doing length and width

In [35]:
# gc = gcloud.Gsheets()

In [36]:
# gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [21]:
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-outdoor - Wipe.csv',index=False) 

In [22]:
looks_good('Bed Bath & Beyond', attribute, df, outdoor_new)